### Rice P&A Journal Club
# ["Detection of nanoflare-heated plasma in the solar corona by the FOXSI-2 sounding rocket"](https://www.nature.com/articles/s41550-017-0269-z)
## Shin-nosuke Ishikawa, Lindsay Glesener, Säm Krucker, Steven Christe, Juan Camilo Buitrago-Casas, Noriyuki Narukage and Juliana Vievering
### Will Barnes, 13 April 2018

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import sunpy.map

%matplotlib inline

# Outline
* The Coronal Heating Problem
* Nanoflares
* FOXSI
* Observations of Hot Plasma

# The Coronal Heating Problem

* Intro coronal heating
* Nice picture of EUV sun versus visual sun
* Movie of flare
* Don't care about flares
* Care about non-flaring ARs

# Nanoflares

* Brief explanation of nanoflares
* A few cartoons
* Explain (briefly) why these are so hard to detect

# Focusing Optics X-ray Solar Imager (FOXSI)

* Discuss the instrument
  * sounding rocket
  * Paper presents results from second flight 
  * FOXSI-3 will fly again in August of this year
* Advantages of HXR versus EUV

<p float="center">
<img src="img/foxsi-1-logo.png" width="300">
<img src="img/foxsi-2-logo.png" width="300">
<img src="img/foxsi-3-logo.png" width="300">
</p>

# Observations of Hot Plasma

* Show figures from paper
* Talk about the results

# Aside: The Differential Emission Measure ($\mathrm{DEM}(T)$)

* Illustration of loop
* EBTEL example movie showing heating input, temperature, density, and DEM
  * Constant heating
  * single pulse

# Observations of Hot Plasma

* Give a bit more context

# TL;DR
* The Sun is *hot*
* Faint hot plasma is hard to observe
* Faint hot plasma is an indicator of many small bursts of energy heating the corona
* Hard X-rays are good for measuring faint hot plasma
* FOXSI measured faint hot plasma
* Many small bursts of energy heating the corona???